In [ ]:
 current_dir = "gdrive/My Drive/Colab Notebooks/Thesis/gold_standard"


import os
import glob
from google.colab import drive
import pandas as pd
import re
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import re

# Mount Google Drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [ ]:
os.chdir(current_dir)

## Features

In [ ]:
df = pd.read_pickle("gold_psy.pkl")

In [ ]:
group_dict = {'ridgeway': 'handbook',
 'bundy': 'handbook',
 'kemper': 'handbook',
 'gacy': 'handbook',
 'dahmer': 'handbook',
 'bernardo': 'cbc',
 'mitchell': 'articles',
 'marsh': 'articles',
 'wayne': 'doc',
 'carl': 'doc'}

In [ ]:
df

name  score                                               text
0     wayne   40.0  \nmy very first contemplation of criminal beha...
1      carl   36.0  I had a statutory rape on a minor fifteen uh t...
2     bundy   34.0  \nyes that's true\n\ni grew up in a wonderful ...
3  bernardo   35.0  \npaul jason teal aka paul bernardo\n\n\nyeah ...
4    dahmer   23.0  \nhey you're lucky you came up on a\n\n\nday w...
5      gacy   27.0  \n\nwell that's hogwash yeah that's strictly h...
6    kemper   26.0  \nwell i'm not an expert i'm not an authority ...
7     marsh   34.5  \nevery time i look at someone i see flashes o...
8  mitchell   34.0  that would be nice i'd like to use the restroo...
9  ridgeway   19.0  \ni didn't really think i had a a urge to kill...

In [ ]:

def clean_text(text):
    # Replace newline characters with a space
    text = text.replace('\n', ' ')

    # Replace multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text)

    # Remove non-alphanumeric characters (but keep single whitespaces)
    text = re.sub(r'[^a-zA-Z0-9 ]', '', text)

    # Convert text to lowercase
    return text.lower()


# Example usage

df['clean_text'] = df['text'].apply(clean_text)

In [ ]:
def two_tier_score(score):
    if score > 30:
        return "psychopathic"
    else:
        return "not psychopathic"

# Apply the function to the 'Score' column
df['binary'] = df['score'].apply(two_tier_score)

In [ ]:
def three_tier_score(score):
    if score >= 30:
        return "high"
    elif 21 <= score < 29:
        return "medium"
    else:
        return "low"

# Apply the function to the 'score' column
df['trinary'] = df['score'].apply(three_tier_score)

In [ ]:
def select_rows_by_group(group_name):
    return df[df['group'] == group_name]

df['group'] = df['name'].map(group_dict)

#Linguistic Features

In [ ]:
df

name  score                                               text  \
0     wayne   40.0  \nmy very first contemplation of criminal beha...   
1      carl   36.0  I had a statutory rape on a minor fifteen uh t...   
2     bundy   34.0  \nyes that's true\n\ni grew up in a wonderful ...   
3  bernardo   35.0  \npaul jason teal aka paul bernardo\n\n\nyeah ...   
4    dahmer   23.0  \nhey you're lucky you came up on a\n\n\nday w...   
5      gacy   27.0  \n\nwell that's hogwash yeah that's strictly h...   
6    kemper   26.0  \nwell i'm not an expert i'm not an authority ...   
7     marsh   34.5  \nevery time i look at someone i see flashes o...   
8  mitchell   34.0  that would be nice i'd like to use the restroo...   
9  ridgeway   19.0  \ni didn't really think i had a a urge to kill...   

                                          clean_text            binary  \
0   my very first contemplation of criminal behav...      psychopathic   
1  i had a statutory rape on a minor fifteen uh t...      psychopathic   
2   yes thats true i grew up in a wonderful home ...      psychopathic   
3   paul jason teal aka paul bernardo yeah im and...      psychopathic   
4   hey youre lucky you came up on a day when the...  not psychopathic   
5   well thats hogwash yeah thats strictly hogwas...  not psychopathic   
6   well im not an expert im not an authority im ...  not psychopathic   
7   every time i look at someone i see flashes of...      psychopathic   
8  that would be nice id like to use the restroom...      psychopathic   
9   i didnt really think i had a a urge to kill h...  not psychopathic   

  trinary     group  
0    high       doc  
1    high       doc  
2    high  handbook  
3    high       cbc  
4  medium  handbook  
5  medium  handbook  
6  medium  handbook  
7    high  articles  
8    high  articles  
9     low  handbook

In [ ]:
!pip install -U spacy
!python -m spacy download en_core_web_sm
import spacy
# Load English tokenizer, tagger, parser and NER
nlp = spacy.load("en_core_web_sm")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.6.1
    Uninstalling spacy-3.6.1:
      Successfully uninstalled spacy-3.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.6.0 requires spacy<3.7.0,>=3.6.0, but you have spacy 3.7.2 which is incompatible.
2023-12-25 00:45:04.249990: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-25 00:45:04.250040: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Att

In [ ]:
import spacy
import pandas as pd

# Load the SpaCy model
nlp = spacy.load("en_core_web_sm")

def pos_tag_dataframe(df):
    """
    Function to tag a DataFrame's text column with POS tags and generate new columns for each tag.

    Parameters:
    df (pandas.DataFrame): The DataFrame containing a text column.

    Returns:
    pandas.DataFrame: The original DataFrame with additional columns for each POS tag.
    """
    pos_tags = ['ADJ', 'ADP', 'ADV', 'AUX', 'CONJ', 'CCONJ', 'DET', 'INTJ', 'NOUN', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM', 'VERB', 'X', 'SPACE']

    for pos_tag in pos_tags:
        # Create a new column with the tag name and initialize it with empty strings
        df[pos_tag] = ''

    # Iterate over the rows and tag each text with POS tags
    for index, row in df.iterrows():
        doc = nlp(row['clean_text'])  # Assuming 'text' is the name of your text column
        for token in doc:
            if token.pos_ in pos_tags:
                # Append the token text to the corresponding column
                df.at[index, token.pos_] += token.text + ' '

    # Remove trailing spaces
    for pos_tag in pos_tags:
        df[pos_tag] = df[pos_tag].str.strip()

    return df

# Example usage:

rich = pos_tag_dataframe(df)



In [6]:
rich.head(1)

NameError: name 'rich' is not defined

In [ ]:
copy = rich

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
import numpy as np

# Text Vectorization (Bag-of-Words)
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(copy['clean_text'])

# Label Encoding
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(copy['binary'])

# Leave-One-Out Cross-Validation
loo = LeaveOneOut()
y_true, y_pred = [], []
confused_samples = []  # List to store confused samples

for train_index, test_index in loo.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Creating and training the SVC model
    model = SVC(kernel='linear')
    model.fit(X_train, y_train)

    # Making predictions
    predictions = model.predict(X_test)

    # Check for misclassification
    if predictions[0] != y_test[0]:
        confused_sample = {
            'Name': copy.iloc[test_index]['name'].values[0],
            'Clean Text': copy.iloc[test_index]['clean_text'].values[0][0:500],
            'True Label': label_encoder.inverse_transform([y_test[0]])[0],
            'Predicted Label': label_encoder.inverse_transform([predictions[0]])[0]
        }
        confused_samples.append(confused_sample)

    # Storing true and predicted values for later use
    y_true.extend(y_test)
    y_pred.extend(predictions)

# Classification Report and Confusion Matrix
print(classification_report(y_true, y_pred, target_names=label_encoder.classes_))


# Printing confused samples
print("\nConfused Samples:")
for sample in confused_samples:
    print(sample)

                  precision    recall  f1-score   support

not psychopathic       0.75      0.75      0.75         4
    psychopathic       0.83      0.83      0.83         6

        accuracy                           0.80        10
       macro avg       0.79      0.79      0.79        10
    weighted avg       0.80      0.80      0.80        10


Confused Samples:
{'Name': 'bundy', 'Clean Text': ' yes thats true i grew up in a wonderful home with two dedicated and loving parents one of five brothers and sisters but as a young a a young boy and i mean the boy of uh twelve or thirteen certainly that i encountered outside the home again in um the local grocery store the local drugstore the soft core pornography what people call soft core um but as i think i i explained to you last night doctor dobson in an anecdote that as young boys do we explore the the back roads and sideways and byways ', 'True Label': 'psychopathic', 'Predicted Label': 'not psychopathic'}
{'Name': 'kemper', 'Clean

# Classification; Pronouns; SVC

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
import numpy as np

# Text Vectorization (Bag-of-Words)
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(copy['pronouns'])

# Label Encoding
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(copy['binary'])

# Leave-One-Out Cross-Validation
loo = LeaveOneOut()
y_true, y_pred = [], []
confused_samples = []  # List to store confused samples

for train_index, test_index in loo.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Creating and training the SVC model
    model = SVC(kernel='linear')
    model.fit(X_train, y_train)

    # Making predictions
    predictions = model.predict(X_test)

    # Check for misclassification
    if predictions[0] != y_test[0]:
        confused_sample = {
            'Name': copy.iloc[test_index]['name'].values[0],
            'Clean Text': copy.iloc[test_index]['clean_text'].values[0][0:500],
            'True Label': label_encoder.inverse_transform([y_test[0]])[0],
            'Predicted Label': label_encoder.inverse_transform([predictions[0]])[0]
        }
        confused_samples.append(confused_sample)

    # Storing true and predicted values for later use
    y_true.extend(y_test)
    y_pred.extend(predictions)

# Classification Report and Confusion Matrix
print(classification_report(y_true, y_pred, target_names=label_encoder.classes_))


# Printing confused samples
print("\nConfused Samples:")
for sample in confused_samples:
    print(sample)


                  precision    recall  f1-score   support

not psychopathic       1.00      0.75      0.86         4
    psychopathic       0.86      1.00      0.92         6

        accuracy                           0.90        10
       macro avg       0.93      0.88      0.89        10
    weighted avg       0.91      0.90      0.90        10


Confused Samples:
{'Name': 'gacy', 'Clean Text': ' well thats hogwash yeah thats strictly hogwash im constantly told that it that i im misquoted constantly saying that i i said this and i said that and its not happened right ive always felt that my attorneys were handling it and and they said to keep a low profile and stay away from it and uh in light of all that has been used against me in the media theyve theyve created this fantasy monster image and its been going on for the last twelve years and ive ever had no comment and i had no uh need ', 'True Label': 'not psychopathic', 'Predicted Label': 'psychopathic'}


## Classification: Pronouns; Logistic Regression

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder

# Text Vectorization (Bag-of-Words)
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(copy['pronouns'])

# Label Encoding
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(copy['binary'])

# Leave-One-Out Cross-Validation
loo = LeaveOneOut()
y_true, y_pred = [], []
confused_samples = []  # List to store confused samples

for train_index, test_index in loo.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Creating and training the Logistic Regression model
    model = LogisticRegression()
    model.fit(X_train, y_train)

    # Making predictions
    predictions = model.predict(X_test)

    # Check for misclassification
    if predictions[0] != y_test[0]:
        confused_sample = {
            'Name': copy.iloc[test_index]['name'].values[0],
            'Clean Text': copy.iloc[test_index]['clean_text'].values[0][0:500],
            'True Label': label_encoder.inverse_transform([y_test[0]])[0],
            'Predicted Label': label_encoder.inverse_transform([predictions[0]])[0]
        }
        confused_samples.append(confused_sample)

    # Storing true and predicted values for later use
    y_true.extend(y_test)
    y_pred.extend(predictions)

# Classification Report and Confusion Matrix
print(classification_report(y_true, y_pred, target_names=label_encoder.classes_))

# Printing confused samples
print("\nConfused Samples:")
for sample in confused_samples:
    print(sample)


                  precision    recall  f1-score   support

not psychopathic       1.00      0.75      0.86         4
    psychopathic       0.86      1.00      0.92         6

        accuracy                           0.90        10
       macro avg       0.93      0.88      0.89        10
    weighted avg       0.91      0.90      0.90        10


Confused Samples:
{'Name': 'gacy', 'Clean Text': ' well thats hogwash yeah thats strictly hogwash im constantly told that it that i im misquoted constantly saying that i i said this and i said that and its not happened right ive always felt that my attorneys were handling it and and they said to keep a low profile and stay away from it and uh in light of all that has been used against me in the media theyve theyve created this fantasy monster image and its been going on for the last twelve years and ive ever had no comment and i had no uh need ', 'True Label': 'not psychopathic', 'Predicted Label': 'psychopathic'}


In [ ]:
df.columns

Index(['name', 'score', 'text', 'clean_text', 'binary', 'trinary', 'group',
       'ADJ', 'ADP', 'ADV', 'AUX', 'CONJ', 'CCONJ', 'DET', 'INTJ', 'NOUN',
       'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM', 'VERB', 'X',
       'SPACE'],
      dtype='object')

0    was was s had would was do be is m are can be ...
1                                was was be is is m be
2    do would re might would is would is will is wa...
3    did was did ve been was did did were can was a...
4    was was was were were was do been been be is i...
5    s m m ve were has been ve ve been ve had were ...
6    has been would be is did did did m is was was ...
7    ca do was should have been do is do do was was...
8    would be may is can have am was did was ll was...
9    did did was being was do was could was did mus...
Name: AUX, dtype: object

## get the loocv perofrmance of a feature

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import LeaveOneOut

def perform_leave_one_out_cv(column_name, data):
    """
    Perform leave-one-out cross-validation on a specified column of a DataFrame.

    Args:
    column_name (str): The name of the column to be used as features.
    data (DataFrame): The DataFrame containing the dataset.

    Returns:
    None: Prints the average Mean Squared Error and R-squared values.
    """

    # Extract features (bag of words) and target variable
    X = data[column_name]
    y = data['score']
    names = data['name']  # Assuming there is a 'name' column in the dataset

    # Initialize variables to store evaluation metrics
    mse_values = []
    r2_values = []

    # Initialize lists to store predicted and actual values, and names
    predicted_values = []
    actual_values = []
    fold_names = []

    # Vectorize the text data using CountVectorizer
    vectorizer = CountVectorizer()

    # Perform leave-one-out cross-validation
    loo = LeaveOneOut()
    for train_index, test_index in loo.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        name = names.iloc[test_index].values[0]  # Get the name for this fold

        # Check if X_train is empty or contains only stop words
        if not X_train.any():
            continue  # Skip to the next iteration if X_train is empty
        else:
            # Vectorize the training and test data
            X_train_vectorized = vectorizer.fit_transform(X_train)
            X_test_vectorized = vectorizer.transform(X_test)

            # Initialize and fit the Support Vector Regression model
            svr = SVR(kernel='linear')
            svr.fit(X_train_vectorized, y_train)

            # Make predictions on the test point
            y_pred = svr.predict(X_test_vectorized)

            # Calculate Mean Squared Error for this fold
            mse = mean_squared_error(y_test, y_pred)

            # Append MSE to the list
            mse_values.append(mse)

            # Append the real and predicted values
            actual_values.append(y_test.values[0])
            predicted_values.append(y_pred[0])

            # Append the name for reference
            fold_names.append(name)

    # Calculate the average MSE over all folds
    average_mse = sum(mse_values) / len(mse_values)

    # Calculate R-squared using the real and predicted values
    r2 = r2_score(actual_values, predicted_values)

    # Print average evaluation metrics
    print(f"Column: {column_name}")
    print(f"Average Mean Squared Error (MSE): {average_mse}")
    print(f"R-squared (R2): {r2}")

    # Print predicted and actual values along with names
    for i in range(len(fold_names)):
        print(f"Fold {i + 1} - Name: {fold_names[i]}, Actual: {actual_values[i]}, Predicted: {predicted_values[i]}")

# Example usage of the function
# df = pd.read_csv('your_data.csv')  # Load your dataset
# perform_leave_one_out_cv('AUX', df)


# Feature Selection: SVR over each fetures alone; leave one out cross validation to get R squared and MSE

In [ ]:
u = ['ADJ', 'ADP', 'ADV', 'AUX', 'CONJ', 'CCONJ', 'DET', 'INTJ', 'NOUN',
       'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM', 'VERB', 'X',
       'SPACE']

In [ ]:
for column in u:
  try:
    perform_leave_one_out_cv(column, data)
  except:
    continue

Column: ADJ
Average Mean Squared Error (MSE): 44.47511151540789
R-squared (R2): -0.10903588343389803
Fold 1 - Name: wayne, Actual: 40.0, Predicted: 31.753475755176844
Fold 2 - Name: carl, Actual: 36.0, Predicted: 32.63615814343991
Fold 3 - Name: bundy, Actual: 34.0, Predicted: 28.84432202031456
Fold 4 - Name: bernardo, Actual: 35.0, Predicted: 28.995357988048102
Fold 5 - Name: dahmer, Actual: 23.0, Predicted: 31.87257122087299
Fold 6 - Name: gacy, Actual: 27.0, Predicted: 30.602268530282487
Fold 7 - Name: kemper, Actual: 26.0, Predicted: 31.80748874887369
Fold 8 - Name: marsh, Actual: 34.5, Predicted: 34.117142351577186
Fold 9 - Name: mitchell, Actual: 34.0, Predicted: 35.23689486344937
Fold 10 - Name: ridgeway, Actual: 19.0, Predicted: 32.2548672868332
Column: ADP
Average Mean Squared Error (MSE): 174.69010241322974
R-squared (R2): -3.35609007950202
Fold 1 - Name: wayne, Actual: 40.0, Predicted: 28.69056619390431
Fold 2 - Name: carl, Actual: 36.0, Predicted: 47.427048770855066
Fold 3 

### function where you can combine features to feed into svr model

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import LeaveOneOut
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

def perform_leave_one_out_cv(data, feature_columns, target_column='score', name_column='name'):
    """
    Perform leave-one-out cross-validation using specified feature columns.

    Args:
    data (DataFrame): The DataFrame containing the dataset.
    feature_columns (list of str): List of column names to be used as features.
    target_column (str): Name of the target column.
    name_column (str): Name of the column containing names for reference.

    Returns:
    None: Prints the average Mean Squared Error and R-squared values.
    """

    # Extract features and target variable
    X = data[feature_columns]
    y = data[target_column]
    names = data[name_column]

    # Initialize variables to store evaluation metrics
    mse_values = []
    r2_values = []

    # Initialize lists to store predicted and actual values, and names
    predicted_values = []
    actual_values = []
    fold_names = []

    # Define the column transformer dynamically
    transformers = [(f'vect_{col}', CountVectorizer(), col) for col in feature_columns]
    column_transformer = ColumnTransformer(transformers=transformers)

    # Define the pipeline with the column transformer and SVR
    pipeline = Pipeline([
        ('transformer', column_transformer),
        ('svr', SVR(kernel='linear'))
    ])

    # Perform leave-one-out cross-validation
    loo = LeaveOneOut()
    for train_index, test_index in loo.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        name = names.iloc[test_index].values[0]

        # Check if X_train is empty
        if X_train.isnull().all().all():
            continue
        else:
            # Fit the pipeline on the training data
            pipeline.fit(X_train, y_train)

            # Make predictions on the test point
            y_pred = pipeline.predict(X_test)

            # Calculate Mean Squared Error for this fold
            mse = mean_squared_error(y_test, y_pred)

            # Append MSE to the list
            mse_values.append(mse)

            # Append the real and predicted values
            actual_values.append(y_test.values[0])
            predicted_values.append(y_pred[0])

            # Append the name for reference
            fold_names.append(name)

    # Calculate the average MSE over all folds
    average_mse = sum(mse_values) / len(mse_values)

    # Calculate R-squared using the real and predicted values
    r2 = r2_score(actual_values, predicted_values)

    # Print average evaluation metrics
    print(f"Columns: {', '.join(feature_columns)}")
    print(f"Average Mean Squared Error (MSE): {average_mse}")
    print(f"R-squared (R2): {r2}")

    # Print predicted and actual values along with names
    for i in range(len(fold_names)):
        print(f"Fold {i + 1} - Name: {fold_names[i]}, Actual: {actual_values[i]}, Predicted: {predicted_values[i]}")

# Example usage
# df = pd.read_csv('your_data.csv')
# feature_cols = ['PRON', 'AUX', 'NUM', 'VERB', 'ADV']
# perform_leave_one_out_cv(df, feature_cols)


### Try out all combos of best performing features: leave one out

In [ ]:
all_combinations = [
    ['PRON'],
    ['AUX'],
    ['NUM'],
    ['VERB'],
    ['ADV'],
    ['PRON', 'AUX'],
    ['PRON', 'NUM'],
    ['PRON', 'VERB'],
    ['PRON', 'ADV'],
    ['AUX', 'NUM'],
    ['AUX', 'VERB'],
    ['AUX', 'ADV'],
    ['NUM', 'VERB'],
    ['NUM', 'ADV'],
    ['VERB', 'ADV'],
    ['PRON', 'AUX', 'NUM'],
    ['PRON', 'AUX', 'VERB'],
    ['PRON', 'AUX', 'ADV'],
    ['PRON', 'NUM', 'VERB'],
    ['PRON', 'NUM', 'ADV'],
    ['PRON', 'VERB', 'ADV'],
    ['AUX', 'NUM', 'VERB'],
    ['AUX', 'NUM', 'ADV'],
    ['AUX', 'VERB', 'ADV'],
    ['NUM', 'VERB', 'ADV'],
    ['PRON', 'AUX', 'NUM', 'VERB'],
    ['PRON', 'AUX', 'NUM', 'ADV'],
    ['PRON', 'AUX', 'VERB', 'ADV'],
    ['PRON', 'NUM', 'VERB', 'ADV'],
    ['AUX', 'NUM', 'VERB', 'ADV'],
    ['PRON', 'AUX', 'NUM', 'VERB', 'ADV']
]


In [ ]:
df = data
for feature_col in all_combinations:
  perform_leave_one_out_cv(df, feature_col)

Columns: PRON
Average Mean Squared Error (MSE): 15.302508671977503
R-squared (R2): 0.6184150945208526
Fold 1 - Name: wayne, Actual: 40.0, Predicted: 34.152102086814864
Fold 2 - Name: carl, Actual: 36.0, Predicted: 40.87588530146328
Fold 3 - Name: bundy, Actual: 34.0, Predicted: 33.49653126332478
Fold 4 - Name: bernardo, Actual: 35.0, Predicted: 31.55997695925472
Fold 5 - Name: dahmer, Actual: 23.0, Predicted: 27.272821023322635
Fold 6 - Name: gacy, Actual: 27.0, Predicted: 31.569874686321867
Fold 7 - Name: kemper, Actual: 26.0, Predicted: 23.82139706929955
Fold 8 - Name: marsh, Actual: 34.5, Predicted: 33.05594696612617
Fold 9 - Name: mitchell, Actual: 34.0, Predicted: 35.1597503456635
Fold 10 - Name: ridgeway, Actual: 19.0, Predicted: 24.970620156185614
Columns: AUX
Average Mean Squared Error (MSE): 10.698771030567853
R-squared (R2): 0.7332143624320715
Fold 1 - Name: wayne, Actual: 40.0, Predicted: 36.42265649614128
Fold 2 - Name: carl, Actual: 36.0, Predicted: 40.01371652229449
Fold 

## Best Performing Combo ['AUX', 'NUM'] ----> 0.5 TTS (IS THIS A METHODOLOGICAL ERROR BECAUSE I SELECTED THESE BY TRAINING LOOCV AND NOW IM DOING TTS?)

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Load your dataset into a DataFrame
data = df

# Extract features and target variable using only 'AUX' and 'NUM' columns
X = data[['AUX', 'NUM']]
y = data['score']
names = data['name']  # Assuming you have a 'name' column in your dataset

# Split data into training and test sets (50% train, 50% test)
X_train, X_test, y_train, y_test, names_train, names_test = train_test_split(
    X, y, names, test_size=0.8, random_state=42)

# Define the column transformer for 'AUX' and 'NUM' columns
column_transformer = ColumnTransformer(
    transformers=[
        ('vect_aux', CountVectorizer(), 'AUX'),
        ('vect_num', CountVectorizer(), 'NUM')
    ]
)

# Define the pipeline with the column transformer and SVR
pipeline = Pipeline([
    ('transformer', column_transformer),
    ('svr', SVR(kernel='linear'))
])

# Fit the pipeline on the training data
pipeline.fit(X_train, y_train)

# Make predictions on the test set
y_pred = pipeline.predict(X_test)

# Calculate Mean Squared Error and R-squared for the test set
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print evaluation metrics
print(f"Columns: AUX, NUM")
print(f"Mean Squared Error (MSE): {mse}")
print(f"R-squared (R2): {r2}")

# Print predicted, actual values, and names
for i in range(len(names_test)):
    print(f"Name: {names_test.iloc[i]}, Actual: {y_test.iloc[i]}, Predicted: {y_pred[i]}")


Columns: AUX, NUM
Mean Squared Error (MSE): 8.027027773719372
R-squared (R2): 0.8217299288564103
Name: mitchell, Actual: 34.0, Predicted: 33.31215574548908
Name: carl, Actual: 36.0, Predicted: 34.21471984805318
Name: gacy, Actual: 27.0, Predicted: 25.79078822412156
Name: wayne, Actual: 40.0, Predicted: 34.14786324786325
Name: marsh, Actual: 34.5, Predicted: 33.11158594491928
Name: bundy, Actual: 34.0, Predicted: 32.46809116809117
Name: ridgeway, Actual: 19.0, Predicted: 22.222317188983858
Name: dahmer, Actual: 23.0, Predicted: 26.19192782526116


## Best Performing Combo ['AUX', 'NUM', 'ADV'] ----> 0.5 TTS

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Load your dataset into a DataFrame
data = df

# Extract features and target variable using 'AUX', 'NUM', and 'ADV' columns
X = data[['AUX', 'NUM', 'ADV']]  # Include 'ADV' column
y = data['score']
names = data['name']  # Assuming you have a 'name' column in your dataset

# Split data into training and test sets (50% train, 50% test)
X_train, X_test, y_train, y_test, names_train, names_test = train_test_split(
    X, y, names, test_size=0.8, random_state=42)

# Define the column transformer for 'AUX', 'NUM', and 'ADV' columns
column_transformer = ColumnTransformer(
    transformers=[
        ('vect_aux', CountVectorizer(), 'AUX'),
        ('vect_num', CountVectorizer(), 'NUM'),
        ('vect_adv', CountVectorizer(), 'ADV')  # Add transformer for 'ADV' column
    ]
)

# Define the pipeline with the column transformer and SVR
pipeline = Pipeline([
    ('transformer', column_transformer),
    ('svr', SVR(kernel='linear'))
])

# Fit the pipeline on the training data
pipeline.fit(X_train, y_train)

# Make predictions on the test set
y_pred = pipeline.predict(X_test)

# Calculate Mean Squared Error and R-squared for the test set
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print evaluation metrics
print(f"Columns: AUX, NUM, ADV")  # Update column list in print statement
print(f"Mean Squared Error (MSE): {mse}")
print(f"R-squared (R2): {r2}")

# Print predicted, actual values, and names
for i in range(len(names_test)):
    print(f"Name: {names_test.iloc[i]}, Actual: {y_test.iloc[i]}, Predicted: {y_pred[i]}")


Columns: AUX, NUM, ADV
Mean Squared Error (MSE): 18.58662734615065
R-squared (R2): 0.5872146611768398
Name: mitchell, Actual: 34.0, Predicted: 31.51077027948194
Name: carl, Actual: 36.0, Predicted: 31.894683026584868
Name: gacy, Actual: 27.0, Predicted: 26.291956373551468
Name: wayne, Actual: 40.0, Predicted: 32.06864349011588
Name: marsh, Actual: 34.5, Predicted: 32.314587593728696
Name: bundy, Actual: 34.0, Predicted: 31.27682344921609
Name: ridgeway, Actual: 19.0, Predicted: 24.42038173142468
Name: dahmer, Actual: 23.0, Predicted: 27.545671438309476
